In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

from transformers import AutoProcessor, LayoutLMv3ForTokenClassification

from mozilla_sec_eia.ex_21.inference import (
    create_inference_dataset,
    perform_inference
)
from mozilla_sec_eia.utils.pdf import (
    draw_boxes_on_img,
    unnormalize_box
)

In [ ]:
has_labels = True

In [ ]:
label_list = ['O', 'B-Subsidiary', 'I-Subsidiary', 'B-Loc', 'I-Loc', 'B-Own_Per']
id2label = {k: v for k,v in enumerate(label_list)}
label2id = {v: k for k,v in enumerate(label_list)}

In [ ]:
# load_model function loads a model with mlflow
model_path = Path("../models/layoutlm_v1_50_labeled_docs")

In [ ]:
model = LayoutLMv3ForTokenClassification.from_pretrained(model_path,
                                                         id2label=id2label,
                                                         label2id=label2id)

In [ ]:
processor = AutoProcessor.from_pretrained(
        "microsoft/layoutlmv3-base", apply_ocr=False
    )

In [ ]:
pdf_dir = Path("../sec10k_filings/pdfs")

In [ ]:
# only necessary if using data with labels
labeled_json_dir = Path("../sec10k_filings/labeled_jsons_v0.1")

# Create a Dataset and Perform Inference

In [ ]:
dataset = create_inference_dataset(
        pdfs_dir=pdf_dir, labeled_json_dir=labeled_json_dir, has_labels=has_labels
    )

In [ ]:
# only use 3 examples
dataset_index = [0, 1, 2]

In [ ]:
# check but I think this is mainly slow because it's checking to make sure PDFs and JSONs are cached
logit_list, pred_list = perform_inference(pdf_dir, model, processor, dataset_index, labeled_json_dir, has_labels)

# Visualize with no labels

In [ ]:
assert not has_labels

In [ ]:
def visual_inputs():
    for i in range(len(pred_list)):
        predictions = pred_list[i]
        example = dataset[i]
        image = example["image"]
        words = example["tokens"]
        boxes = example["bboxes"]
        encoding = processor(
            image,
            words,
            boxes=boxes,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
        )
        yield predictions, encoding, image

gen = visual_inputs()

In [ ]:
predictions, encoding, image = next(gen)
width, height = image.size
preds = [model.config.id2label[pred] for pred in predictions]
token_boxes = encoding.bbox.squeeze().tolist()
boxes = [unnormalize_box(box, width, height) for box in token_boxes]

image = image.convert("RGB")
draw_boxes_on_img(preds, boxes, image, width, height)
image

# Visualize with Labels

In [ ]:
assert has_labels

In [ ]:
def convert_ner_tags_to_id(ner_tags):
    return [int(label2id[ner_tag]) for ner_tag in ner_tags]

def visual_inputs_with_labels():
    for i in range(len(pred_list)):
        predictions = pred_list[i]
        example = dataset[i]
        image = example["image"]
        words = example["tokens"]
        boxes = example["bboxes"]
        ner_tags = convert_ner_tags_to_id(example["ner_tags"])
        encoding = processor(
            image,
            words,
            boxes=boxes,
            word_labels=ner_tags,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
        )
        yield predictions, encoding, image

gen_w_labels = visual_inputs_with_labels()

In [ ]:
# TODO: also add visualizaton for wrong predictions
predictions, encoding, image = next(gen_w_labels)
width, height = image.size
token_boxes = encoding.bbox.squeeze().tolist()
boxes = [unnormalize_box(box, width, height) for box in token_boxes]
labels = encoding.labels.squeeze().tolist()

true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]
true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

image = image.convert("RGB")
draw_boxes_on_img(true_predictions, true_boxes, image, width, height)
image